# Building an ETL Pipeline for WRC Hackathon

### Tools

In [10]:
# Importing Packages

import tweepy as tw         
import pandas as pd     
import numpy as np
from datetime import date, timedelta
import json
import re
from IPython.display import display
import pyodbc

In [25]:
# API Keys

CONSUMER_KEY    = '9qH6416zLY2GENYQzvkeFyP2o'
CONSUMER_SECRET = '8XOo0ENIQy4HjX5jXC1xSdfj7jiOqubesglT29QsKkmHaDtEeh'

ACCESS_TOKEN  = '1219722321289777164-hlhI8xizj3M39DU6VdVhsPyQuAWbgF'
ACCESS_SECRET = 'l2WyzCqaaUfadV9P4N53zjJ3OOvqUb0IxNskk2GyOAiMM'

In [26]:
# Create authentication

auth = tw.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tw.API(auth, wait_on_rate_limit=True)

### Data

Both sets need expansion

In [27]:
 key_words = [ 'leak' , 'burst' , 'pressure', 'water','cut', 'pipe',
              'fault', 'broke', 'damage', 'toilet', 'sanit',
              'waste', 'complain', 'supply','service','deliver',
              'out', 'no', 'reference','report' ,'complain',
              'install','low','repair']
#Words like flow included in 'low','broken' in 'broke' (Pseudo lemmatization)
#Includes word like follow..
#Still to consider outage vs out --using out for now
#sanitation vs sanit (sanitory...)

In [28]:
# master_list =[['alfred_enzo' ,'@alfrednzoDM', 'eastern_cape'],
#               ['amajuba' ,'@AmajubaDM' , 'kwazulu_natal' ],
#               ['cape_wine_lands','@CWDM2','western_cape'],
#               ['ehlanzeni' ,'@EhlanzeniM','mpumalanga'],
#               ['fezile_dabi', '@FezileDabiDM','free_state' ],
#               ['garden_route', '@GardenRoute_DM','western_cape'],
#               ['gert_sibande','@GertSibandeDM','mpumalanga' ],
#               ['ilembe' ,'@DistrictIlembe','kwazulu_natal'],
#               ['king_cetshwayo','@Cetshwayo_dm','kwazulu_natal'],
#               ['lejweleputswa', '@LejweleputswaM','free_state' ],
#               ['mangaung', '@MangaungCity','free_state' ],
#               ['nkangala', '@NkangalaDM','mpumalanga' ],
#               ['sedibeng', '@SedibengDM','gauteng' ],
#               ['ugu', '@UguDM','kwazulu_natal'],
#               ['north_uthungulu','@UmkhanyakudeM','kwazulu_natal'],
#               ['uthukela', '@OkhahlambaLM','kwazulu_natal'],
#               ['vhembe', '@VhembeDM','limpopo'],
#               ['rand_west_city', '@RandLocal','gauteng' ],
#               ['sedibeng' ,'@EmfuleniLM','gauteng' ],
#               ['amathole','@Amathole_DM', 'eastern_cape'] , 
#               [ 'bojanala','Bojanala District', 'north_west'], 
#               [ 'capricon' ,'@CDMunicipality 'limpopo'], 
#               ['karoo' ,'Central Karoo District', 'western_cape'],
#               ['chris_hani' ,'@ChrisHaniDM', 'eastern_cape'], 
#               ['kenneth_kaunda','Dr Kenneth Kaunda District', 'north_west'], 
#               ['dr_ruth','Dr Ruth Segomotsi Mompati District', 'north_west'] ,
#               ['frances','@FrancesBaardDM', 'northern_cape'],
#               ['harry_gwala','Harry Gwala District', 'kwazulu_natal'], 
#               ['joe_gqabi','Joe Gqabi District', 'eastern_cape' ],
#               ['john_taolo','@JTGMunicipal', 'northern_cape'], 
#               ['mopani','Mopani District', 'limpopo'],
#               ['namakwa','Namakwa District', 'northern_Cape'], 
#               ['ngaka','Ngaka Modiri Molema', 'north_west'], 
#               ['or_tambo' ,'@ortambodm', 'eastern_cape'], 
#               ['overberge','Overberg District', 'western_cape'],
#               ['pixley_ka_seme','Pixley ka Seme District', 'northern_cape'], 
#               ['sarah', 'Sarah Baartman District', 'eastern_cape'],
#               ['sekhukune','@SEKHUKHUNEMUNI1', 'limpopo'],
#               ['thabo','@TeamTMDM', 'free_state'] ,
#               ['umgungundlovu' ,'uMgungundlovu District', 'kwazulu_natal'] ,
#               ['umzinyathi','@umzinyathi', 'kwazulu_natal'],
#               ['waterberg','Waterberg District', 'limpopo' ],
#               ['west_cost' ,'@WeskusDM', 'western_cape'], 
#               ['zf_mchunu' ,'ZF Mgcawu District', 'northern_cape'],
#               ['xhariep' ,'Xhariep District', 'free_state'],
#               ['zululand' ,'Zululand District', 'kwazulu_natal']]
master_list = [['','@thameswater','']]

### Functions

In [38]:
def extract_tweets(search_word) :

    search_words = search_word + " -filter:retweets"
    today = date.today()                     
    tweets = tw.Cursor(api.search,
               q=search_words,
               lang="en"
               ).items(200)                    
    tweet_status = [tweet for tweet in tweets]
    tweet_list = [tweet.text for tweet in tweet_status]
    tweet_date = [tweet.created_at for tweet in tweet_status]
    location_1 = [tweet.user.location for tweet in tweet_status]
#     co_od = [tweet.coordinates for tweet in tweet_status]
    place_name = [tweet.place.full_name if tweet.place else 'Not available' for tweet in tweet_status]

    tweet_df = pd.DataFrame(data=tweet_list, 
        columns=['tweet'])
    tweet_df['timestamp'] = tweet_date
#     tweet_df['coordinates'] = co_od
    tweet_df['Place_name'] = place_name
    tweet_df['loc_1'] = location_1
        
    return tweet_df

In [39]:
def TweetCleaner(tweet):
    tweet = str(tweet.lower()) 
    tweet = re.sub('@[\w]*','',tweet) 
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
#     tweet = re.sub(r'#\w*', '', tweet) might put issue in hashtag
    tweet = re.sub(r"[,.;':@#?!\&/$]+\ *", ' ', tweet)
    tweet = re.sub(r"U+FFFD ", ' ', tweet)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    tweet = tweet.lstrip(' ')
    tweet = ' '.join([i for i in tweet.split() if any(list_word in i for list_word in key_words)])  
    return tweet

In [40]:
def create_df(the_list) :
    col_order = ['tweet', 'timestamp','keywords' ,'Place_name','loc_1']   
    dummy_df = pd.DataFrame(columns = ['tweet', 'timestamp','keywords' ,'Place_name','loc_1'])

    for i in the_list :
        handle = i[1]
        df = extract_tweets(handle)
        df['keywords'] = df['tweet'].map(lambda x: TweetCleaner(x))
        df = df[col_order]

        dummy_df = dummy_df.append(df)
    
    dummy_df = dummy_df.drop([0])
    dummy_df = dummy_df.reset_index()
    dummy_df = dummy_df.drop(columns = ['index'] , axis = 1)   
    
    return dummy_df


### Build Data

In [41]:
database_new = create_df(master_list)

In [42]:
database_new

,tweet,timestamp,keywords,Place_name,loc_1
0,"@thameswater We will have a look when we can ,...",2020-08-06 12:52:42,,Not available,
1,@thameswater Hi Daisy it is the broadway Winch...,2020-08-06 12:49:47,now,Not available,
2,@thameswater Thanks but eventually got through,2020-08-06 12:49:37,,Not available,"Richmond, London"
3,Another new @LifeatBromford Sewerage Treatment...,2020-08-06 12:47:13,another delivery,Not available,Cirencester UK
4,@thameswater there are still leaks in Red Lion...,2020-08-06 12:29:32,leaks,Not available,
...,...,...,...,...,...
194,"@thameswater hi there, I still haven’t been cr...",2020-08-04 15:01:25,,Not available,"London, England"
195,@thameswater So newham council don’t want to h...,2020-08-04 14:59:31,no,Not available,Plaistow london
196,@thameswater I’ve constantly got onto the coun...,2020-08-04 14:56:13,water know,Not available,Plaistow london
197,@thameswater it is dangerous on Starts Hill. I...,2020-08-04 14:52:16,no,Not available,


### Database connection

In [44]:
[(i,v) for i,v in enumerate(database_new.Place_name) if v != 'Not available']

[(5, 'Bourton-on-the-Water, England'),
 (33, 'South East, England'),
 (34, 'St. Mary Magdalene, Boveney'),
 (81, 'Basingstoke, England'),
 (83, 'Tottenham, London'),
 (91, 'Cheltenham, England'),
 (92, 'Cheltenham, England'),
 (114, 'Enfield, London'),
 (135, 'Islington, London'),
 (150, 'Tottenham, London'),
 (165, 'Slough, South East'),
 (171, 'The Mousetrap Inn')]

In [45]:
[(i,v) for i,v in enumerate(database_new.loc_1) if v]

[(2, 'Richmond, London'),
 (3, 'Cirencester UK'),
 (5, 'Bourton on the water'),
 (6, 'Chislehurst, Kent'),
 (7, 'London, England'),
 (11, 'Harpenden, Herts '),
 (12, 'Wandsworth, London'),
 (13, '#digitalnomad'),
 (15, 'Liverpool - Sheffield'),
 (16, 'Chislehurst'),
 (17, 'Wiltshire, UK'),
 (18, 'Wiltshire, UK'),
 (19, 'Wiltshire, UK'),
 (20, 'Station Road, RWB'),
 (22, 'Wiltshire, UK'),
 (23, 'Here'),
 (24, 'London'),
 (25, 'Here'),
 (27, 'London'),
 (28, 'England'),
 (29, 'UK, Australia, the Philippines'),
 (30, 'London, England'),
 (31, 'London'),
 (32, 'Helping food brands get sales'),
 (33, 'Windsor, Berkshire'),
 (34, 'Windsor, Berkshire'),
 (36, 'Hackney London N16'),
 (42, 'london, uk'),
 (44, 'London, UK'),
 (45, 'Gloucestershire'),
 (46, 'Richmond, London'),
 (47, '#digitalnomad'),
 (48, 'Liverpool - Sheffield'),
 (49, 'Wiltshire, UK'),
 (50, 'Liverpool - Sheffield'),
 (51, 'Wiltshire, UK'),
 (52, 'Liverpool - Sheffield'),
 (53, 'Wiltshire, UK'),
 (54, 'London'),
 (55, 'Oxfor